In [ ]:
from flask import Flask
from flask import request
import nltk.stem.porter as stemmer
import json
import requests

app = Flask(__name__)


@app.route("/search", methods=["POST"])
def search():
    j = request.json
    query = j['query']
    
    response_analyze = requests.post('http://127.0.0.1:13533/analyze', json={'data' : query})
    parsed_analyze = json.loads(response_analyze.text)
    
    response_reverceindex = requests.post('http://127.0.0.1:13538/reverceindex', json={'data': parsed_analyze['words']})
    parsed_reverceindex = json.loads(response_reverceindex.text)
    
    response_ranked_docs = requests.post('http://127.0.0.1:13541/rank', json={'data': parsed_reverceindex['processed_data']})
    parsed_ranked_docs = json.loads(response_ranked_docs.text)
    
    response_docs_snippets = requests.post('http://127.0.0.1:13542/snippets', json={'data': parsed_ranked_docs['ranked_docs']})
    parsed_docs_snippets = json.loads(response_docs_snippets.text)
    
    return json.dumps({"status":"ok", "processed_data" : parsed_docs_snippets['snippets']})
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13565)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:13565/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Nov/2018 00:20:09] "POST /search HTTP/1.1" 200 -
